# Generate User Preference Data

In [1]:
from google.cloud import secretmanager
import pandas as pd
import numpy as np
from currensee.utils.db_utils import create_pg_engine
from sqlalchemy import text

import pandas as pd
import random
import os
import re
from faker import Faker


2025-07-05 23:13:31,029 - currensee.core.secrets - INFO - PROJECT_ID not found, attempting to load from .env file
2025-07-05 23:13:31,032 - currensee.core.secrets - INFO - Loaded environment from ./.env
2025-07-05 23:13:31,034 - currensee.core.secrets - INFO - SecretManager initialized with project_id: adsp-34002-on02-sopho-scribe


In [2]:
# Define DB_NAME
DB_NAME = "crm_outlook"

In [3]:
engine = create_pg_engine(db_name=DB_NAME)

In [4]:
df1 = pd.read_sql("SELECT * FROM meeting_data", con=engine)
df1.head()

,meeting_timestamp,host,host_email,invitees,invitee_emails,meeting_subject
0,2018-07-17 15:00:00,Jane Moneypenny,jane.moneypenny1@outlook.com,Cynthia Hobbs,cynthia.hobbs@abbvie.com,AbbVie - Introductory Call & Bankwell Services...
1,2018-07-24 14:00:00,Jane Moneypenny,jane.moneypenny1@outlook.com,Jennifer Phelps,jennifer.phelps@aerovironment.com,AeroVironment - Initial Discussion on Financia...
2,2018-08-07 10:00:00,Jane Moneypenny,jane.moneypenny1@outlook.com,Kyle Waters,kyle.waters@amedisys.com,Amedisys - Discussing Cash Management Needs
3,2018-08-15 11:00:00,Jane Moneypenny,jane.moneypenny1@outlook.com,Kyle Waters,kyle.waters@amedisys.com,Amedisys - Cash Concentration Services Demo
4,2018-08-29 14:00:00,Jane Moneypenny,jane.moneypenny1@outlook.com,Denise Moore,denise.moore@celestica.com,Celestica - Initial Discussion on Liquidity Ma...


In [5]:
df1['meeting_timestamp'].unique()

array(['2018-07-17 15:00:00', '2018-07-24 14:00:00',
       '2018-08-07 10:00:00', '2018-08-15 11:00:00',
       '2018-08-29 14:00:00', '2018-09-25 10:00:00',
       '2018-10-11 16:00:00', '2018-11-05 16:00:00',
       '2018-11-14 11:00:00', '2018-12-12 13:00:00',
       '2019-01-30 14:00:00', '2019-02-26 09:00:00',
       '2019-03-05 13:00:00', '2019-03-20 14:30:00',
       '2019-04-10 14:00:00', '2019-04-24 11:30:00',
       '2019-05-17 16:00:00', '2019-07-30 14:00:00',
       '2019-09-17 10:00:00', '2019-10-21 14:00:00',
       '2019-10-23 14:00:00', '2019-10-24 11:00:00',
       '2020-01-28 14:00:00', '2020-01-29 11:00:00',
       '2020-03-09 10:00:00', '2020-04-07 10:00:00',
       '2020-04-21 14:00:00', '2020-04-30 16:00:00',
       '2020-04-27 14:00:00', '2020-07-07 12:00:00',
       '2020-07-22 16:00:00', '2020-07-29 14:00:00',
       '2020-07-30 11:00:00', '2020-09-08 15:00:00',
       '2020-09-15 14:00:00', '2020-10-29 14:00:00',
       '2020-10-30 11:00:00', '2021-01-26 14:0

In [6]:
df_result = pd.read_sql("SELECT * FROM Employees", con=engine)
df_result.head()

,employee_id,first_name,last_name,title,email,phone,hire_date,department,market
0,31e735f6-cfbe-4107-8aba-4ef45a90e9ae,Jane,Moneypenny,Relationship Manager,jane.moneypenny1@bankwell.com,971.841.4711x92417,2021-01-14,Enterprise Investment,San Fransisco
1,b61a7a99-791a-4b9f-923f-271810defac4,Monica,Melton,Product Specialist,monica.melton@bankwell.com,884.541.7837,2021-12-03,Small Business Investment,Chicago
2,2f8c004d-e0a6-4bba-956e-2b60af632674,Michael,Long,Finance Assistant,michael.long@bankwell.com,478-831-1994x378,2021-12-06,Sales,Washington DC
3,59dcd404-9d5c-47f4-9ba1-2de5156343f8,Breanna,Guerra,Senior Relationship Manager,breanna.guerra@bankwell.com,736-597-4792x589,2024-07-11,Enterprise Investment,Boston
4,7a650dd8-34a3-4148-a7f8-631bf5db37d4,Heather,Singh,Financial Advisor,heather.singh@bankwell.com,7139112404,2020-11-15,Customer Support,Washington DC


In [7]:
df_result.shape

(100, 9)

In [8]:
df = df_result[["email", "first_name", "last_name", "title"]]

In [9]:
df.head()

,email,first_name,last_name,title
0,jane.moneypenny1@bankwell.com,Jane,Moneypenny,Relationship Manager
1,monica.melton@bankwell.com,Monica,Melton,Product Specialist
2,michael.long@bankwell.com,Michael,Long,Finance Assistant
3,breanna.guerra@bankwell.com,Breanna,Guerra,Senior Relationship Manager
4,heather.singh@bankwell.com,Heather,Singh,Financial Advisor


In [13]:
df['title'].unique()

array(['Relationship Manager', 'Product Specialist', 'Finance Assistant',
       'Senior Relationship Manager', 'Financial Advisor'], dtype=object)

In [39]:
# Add in columns for initial preferences:
# details on client financial holdings, 
# Latest News Headlines
# Macro Economic Trends
# Past Meeting and Client Conversations

def generate_employee_preference_data(employees_df):
 #   employees_df['finance_pref'] = np.where(employees_df['title'] == 'Relationship Manager', 'True', 
  #                                         np.where(employees_df['title'] == 'Product Specialist', 'False',
  #                                         np.where(employees_df['title'] == 'Finance Assistant', 'False',   
  #                                         np.where(employees_df['title'] == 'Senior Relationship Manager', 'True',            
  #                                         np.where(employees_df['title'] == 'Financial Advisor', 'True', 'True'
  #                                                 )))))
    
    preferences = []
    dt_lst=['2023-01-23 10:00:00', '2023-04-02 10:00:00', '2023-07-26 14:00:00', '2023-11-04 10:00:00', '2024-01-30 10:00:00', '2024-03-10 09:00:00']
    for date in dt_lst:
        for _, employee in employees_df.iterrows():
            client_news_dtl = random.choice([ "full", 'none'])
            finance_pref = random.choice(["full", "short", "none"])
            macro_trend = random.choice([ "full", "none"])
            past_meeting = random.choice(["full", "short", 'none'])
            
            preferences.append(
                {
                    "as_of_date": date,
                    "employee_first_name": employee["first_name"],
                    "employee_last_name": employee["last_name"],
                    "email": employee['email'],
                    "title": employee["title"],
                    "finance_detail": finance_pref,
                    "news_detail": client_news_dtl,
                    "macro_news_detail": macro_trend,
                    "past_meeting_detail":past_meeting
                }
                )
            
    return pd.DataFrame(preferences)

In [40]:
preferences_df = generate_employee_preference_data(df)

In [41]:

# Condition: manually set for janne
condition1 = (preferences_df['employee_first_name'] == 'Jane') & (preferences_df['employee_last_name'] == 'Moneypenny') & (preferences_df['as_of_date']=="2023-01-23 10:00:00")
preferences_df.loc[condition1, ['finance_detail', 'news_detail', 'macro_news_detail', 'past_meeting_detail' ]] = ['full', 'full', 'full', 'full']

condition2 = (preferences_df['employee_first_name'] == 'Jane') & (preferences_df['employee_last_name'] == 'Moneypenny') & (preferences_df['as_of_date']=="2023-04-02 10:00:00")
preferences_df.loc[condition2, ['finance_detail', 'news_detail', 'macro_news_detail', 'past_meeting_detail' ]] = ['short', 'full', 'full', 'short']

condition3 = (preferences_df['employee_first_name'] == 'Jane') & (preferences_df['employee_last_name'] == 'Moneypenny') & (preferences_df['as_of_date']=="2023-07-26 14:00:00")
preferences_df.loc[condition3, ['finance_detail', 'news_detail', 'macro_news_detail', 'past_meeting_detail' ]] = ['none', 'none', 'full', 'short']

condition4 = (preferences_df['employee_first_name'] == 'Jane') & (preferences_df['employee_last_name'] == 'Moneypenny') & (preferences_df['as_of_date']=="2023-11-04 10:00:00")
preferences_df.loc[condition4, ['finance_detail', 'news_detail', 'macro_news_detail', 'past_meeting_detail' ]] = ['short', 'full', 'none', 'none']

condition5 = (preferences_df['employee_first_name'] == 'Jane') & (preferences_df['employee_last_name'] == 'Moneypenny') & (preferences_df['as_of_date']=="2024-01-30 10:00:00")
preferences_df.loc[condition5, ['finance_detail', 'news_detail', 'macro_news_detail', 'past_meeting_detail' ]] = ['short', 'full', 'none', 'full']

condition6 = (preferences_df['employee_first_name'] == 'Jane') & (preferences_df['employee_last_name'] == 'Moneypenny') & (preferences_df['as_of_date']=="2024-03-10 09:00:00")
preferences_df.loc[condition6, ['finance_detail', 'news_detail', 'macro_news_detail', 'past_meeting_detail' ]] = ['full', 'none', 'full', 'full']

preferences_df.loc[preferences_df['employee_first_name'] == 'Jane']

,as_of_date,employee_first_name,employee_last_name,email,title,finance_detail,news_detail,macro_news_detail,past_meeting_detail
0,2023-01-23 10:00:00,Jane,Moneypenny,jane.moneypenny1@bankwell.com,Relationship Manager,full,full,full,full
100,2023-04-02 10:00:00,Jane,Moneypenny,jane.moneypenny1@bankwell.com,Relationship Manager,short,full,full,short
200,2023-07-26 14:00:00,Jane,Moneypenny,jane.moneypenny1@bankwell.com,Relationship Manager,none,none,full,short
300,2023-11-04 10:00:00,Jane,Moneypenny,jane.moneypenny1@bankwell.com,Relationship Manager,short,full,none,none
400,2024-01-30 10:00:00,Jane,Moneypenny,jane.moneypenny1@bankwell.com,Relationship Manager,short,full,none,full
500,2024-03-10 09:00:00,Jane,Moneypenny,jane.moneypenny1@bankwell.com,Relationship Manager,full,none,full,full


In [42]:
preferences_df.to_sql("preferences", engine, if_exists="replace", index=False)

600

In [43]:
alter_sql = """
ALTER TABLE "preferences"
ADD PRIMARY KEY ("as_of_date", "employee_last_name", "employee_first_name" );
"""

with engine.connect() as connection:
    connection.execute(text(alter_sql))

In [45]:
pd.read_sql("SELECT * FROM preferences limit 10", con=engine)

,as_of_date,employee_first_name,employee_last_name,email,title,finance_detail,news_detail,macro_news_detail,past_meeting_detail
0,2023-01-23 10:00:00,Jane,Moneypenny,jane.moneypenny1@bankwell.com,Relationship Manager,full,full,full,full
1,2023-01-23 10:00:00,Monica,Melton,monica.melton@bankwell.com,Product Specialist,full,none,none,none
2,2023-01-23 10:00:00,Michael,Long,michael.long@bankwell.com,Finance Assistant,full,full,none,full
3,2023-01-23 10:00:00,Breanna,Guerra,breanna.guerra@bankwell.com,Senior Relationship Manager,full,full,none,none
4,2023-01-23 10:00:00,Heather,Singh,heather.singh@bankwell.com,Financial Advisor,short,full,none,short
5,2023-01-23 10:00:00,Alejandro,Walters,alejandro.walters@bankwell.com,Senior Relationship Manager,short,full,full,none
6,2023-01-23 10:00:00,Ryan,Holloway,ryan.holloway@bankwell.com,Relationship Manager,none,none,full,none
7,2023-01-23 10:00:00,Anthony,Reynolds,anthony.reynolds@bankwell.com,Financial Advisor,none,none,none,full
8,2023-01-23 10:00:00,Alexandra,Terry,alexandra.terry@bankwell.com,Product Specialist,short,full,full,none
9,2023-01-23 10:00:00,Jordan,Robles,jordan.robles@bankwell.com,Financial Advisor,full,none,none,short


In [46]:
pd.read_sql("SELECT * FROM preferences where email = 'jane.moneypenny1@bankwell.com' ", con=engine)

,as_of_date,employee_first_name,employee_last_name,email,title,finance_detail,news_detail,macro_news_detail,past_meeting_detail
0,2023-01-23 10:00:00,Jane,Moneypenny,jane.moneypenny1@bankwell.com,Relationship Manager,full,full,full,full
1,2023-04-02 10:00:00,Jane,Moneypenny,jane.moneypenny1@bankwell.com,Relationship Manager,short,full,full,short
2,2023-07-26 14:00:00,Jane,Moneypenny,jane.moneypenny1@bankwell.com,Relationship Manager,none,none,full,short
3,2023-11-04 10:00:00,Jane,Moneypenny,jane.moneypenny1@bankwell.com,Relationship Manager,short,full,none,none
4,2024-01-30 10:00:00,Jane,Moneypenny,jane.moneypenny1@bankwell.com,Relationship Manager,short,full,none,full
5,2024-03-10 09:00:00,Jane,Moneypenny,jane.moneypenny1@bankwell.com,Relationship Manager,full,none,full,full
